<a href="https://colab.research.google.com/github/Rpvermaak/hogwan-checker/blob/main/hogwan-cheker/notebooks/Scrape_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Scrape the website "https://greenlist.tokyojon.com/", extract the names following the word "subject:", and save the results in a CSV file.

## Scrape the website and extract data

### Subtask:
Modify the `scrape_website` function to download the webpage, parse the HTML content, and extract the names following "subject:".


In [ ]:
from google.colab import auth
auth.authenticate_user()

print('Authenticated')


Authenticated


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import bigquery
from google.colab import auth

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_website(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    names = []
    # Find all text content and search for "Subject:"
    # A more robust approach is to find elements containing the text "Subject:" and extract the following text.
    for strong_tag in soup.find_all('strong'):
        if "Subject:" in strong_tag.get_text():
            # Extract the text after "Subject:" within the same element or the next sibling
            text_after_subject = strong_tag.get_text().split("Subject:", 1)[1].strip()
            names.append(text_after_subject)

    # Additionally, check for "Subject:" in other elements like 'p' tags as observed previously.
    for p_tag in soup.find_all('p'):
        text = p_tag.get_text()
        start_index = 0
        while True:
            start_index = text.find("Subject:", start_index)
            if start_index == -1:
                break
            # Extract the name after "subject:"
            name_start = start_index + len("Subject:")
            # Find the end of the name (assuming it ends with a newline or the end of the text)
            name_end = text.find('\n', name_start)
            if name_end == -1:
                name = text[name_start:].strip()
            else:
                name = text[name_start:name_end].strip()
            if name and name not in names:  # Only add non-empty and unique names
                names.append(name)
            start_index = name_start + len("Subject:") # Continue searching from after the found subject


    return names

url = "https://greenlist.tokyojon.com/"
names_list = scrape_website(url)
df_names = pd.DataFrame(names_list, columns=['Subject Name'])

# Save the DataFrame to a CSV file
df_names.to_csv('subject_names_green.csv', index=False)

display(df_names.head(10))

,Subject Name
0,Francis Parker Guridasan campus
1,Broadwic English Academy-Mokpo Branch
2,Piaget Kids Academy Gangdong-gu Seoul
3,SIE.K
4,PLATO
5,Francis Parker Bundang
6,Sharing My Experience at Knox Pyeongchon
7,Inje English Village
8,"Knox School, Pyeongchon"
9,"WYATT Institute, Bundang"


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def scrape_website(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    names = set() # Use a set to store unique names efficiently

    # Search for "Subject:" in various common text-holding tags
    tags_to_check = ['p', 'div', 'span', 'li', 'td', 'strong']

    for tag_name in tags_to_check:
        for tag in soup.find_all(tag_name):
            text = tag.get_text()
            # Use regex to find all occurrences of "Subject:" followed by text
            # This regex looks for "Subject:", then captures any characters non-greedily
            # until a newline, another "Subject:", or the end of the string.
            # It also handles potential spaces after "Subject:".
            for match in re.finditer(r"Subject:\s*(.*?)(?=\nSubject:|\n|$)", text, re.IGNORECASE | re.DOTALL):
                name = match.group(1).strip()
                if name:
                    # Take the first line if there are multiple lines (e.g., name followed by comments on new line)
                    name = name.split('\n')[0].strip()
                    if name:
                        names.add(name)

    return sorted(list(names))

url = "https://blacklist.tokyojon.com/"
names_list = scrape_website(url)
df_names = pd.DataFrame(names_list, columns=['Subject Name'])

# Save the DataFrame to a CSV file
df_names.to_csv('subject_names_black.csv', index=False)

print(f"Total names extracted: {len(names_list)}")
display(df_names.head(20))

Total names extracted: 158


,Subject Name
0,ALTIORA GIMHAE / YULHA CAMPUS
1,AMJ Academy Seongnam
2,American STEM Prep (ASP) in Yongin
3,American STEM Prep - Yongin
4,American STEM Prep Kinder Gwanggyo Campus (San...
5,American STEM Prep Yongin
6,American STEM Prep. Songdo
7,American Stem Prep - Yongin
8,American Stem Prep Songdo
9,American Stem Prep in Songdo
